In [1]:
import numpy as np
import numpy.linalg as la

In [2]:
def coupled_flutter_response(mu,sigma,V, a,b,e,r, mode='Steady - Basic'):
    """
    Compute the eignvalues and eigenvectors for anlaysis of the
    flutter response of a coupled system with the given parameters.

    Parameters:
    mu (float): Mass Ratio.
    sigma (float): Frequency Ratio.
    V (float): Reduced Velocity.
    a (float): Torsional Axis Location.
    b (float): Semi-Chord Length.
    e (float): Eccentricity - Center of Mass Location from torsional axis.
    r (float): Radius of Gyration.
    mode (string): Approximation Chosen for Aerodynamic Influence Matrix; 'Steady', 'Unsteady', 'Quasi-Steady' or 'kussner'. - kussner stc

    Returns:
    Eigen Value Problem Solution: 
    Eigenvalues and Eigenvectors to problem below:

    p**2*I - M*^-1(K* - F) = 0
    """

    # Torsional Axis Offset
    x_theta = (e - a) * b

    # Mass and Stiffness Matrices
    M = np.array([[1, x_theta],
                  [x_theta, r**2]])
    
    K = np.array([
        [sigma**2 / V**2, 2/mu],
        [0, (r**2 / V**2) - ((2 / mu) * (a + 0.5))]])
    
    if mode == 'Steady - Basic':

        # Solve the eigenvalue problem
        M_inv = np.linalg.inv(M)
        A = np.matmul(M_inv, K)
        eigvals, eigvecs = np.linalg.eig(-A)

        return eigvals, eigvecs
    
    if mode == 'Steady - State Space':
        #w_theta = 100 # Frequency of Torsional Vibrations, Hz
        # Create state space representation then solve the eigenvalue problem
        A = np.block([
            [np.zeros_like(M), np.eye(2)],
            [la.inv(M) @ (K), np.zeros_like(M)]
        ])

        eigvals, eigvecs = la.eig(A)

        # p,eigvecs = np.linalg.eig(A)

        # eigvals = p * V**2 * w_theta

        # # Remove complex conjugates, retain positive frequencies i.e., p > 0
        # eigvals = eigvals[np.where(np.real(eigvals) > 0)]
        # eigvecs = eigvecs[:, np.where(np.real(eigvals) > 0)]


        # # One big debug print with all the matrices and results
        # print(f"Mass Matrix: {M}")
        # print(f"Stiffness Matrix: {K}")
        # print(f"State Space Matrix: {A}")
        # print(f"Eigenvalues: {eigvals}")
        # print(f"Eigenvectors: {eigvecs}")
        return eigvals, eigvecs
    
    if mode == 'Steady - SHM':
        # Mass and stiffness matrices
        K = np.array([[sigma**2 / V**2, 0],
                    [0, r**2 / V**2]])
        
        F = np.array([[0, -2 * np.pi],
                     [0, -2 * np.pi * (a + 0.5)]])
        # Construct the system matrix
        A = np.block([
            [np.zeros_like(M), np.eye(2)],
            [-la.inv(M) @ (K - F), np.zeros_like(M)]
        ])

        # Compute eigenvalues
        eigvals, eigvecs = la.eig(A)

        return eigvals, eigvecs

    else:
        raise ValueError("Invalid mode selected; choose 'Steady - Basic' or 'Steady - State Space'.")

In [28]:
# -- User definitions -------------------------------------------------------
# Define model parameters (Values from Hodges P.186)
# b = 0.5     #Semi-chord
# e = -0.1    #Distance centre of mass [-] (fraction semi-chord)
# a = -0.2    #Distance torsional axis [-] (fraction semi-chord)
# mu = 20*np.pi     #Mass ratio [-]
# sigma = 0.4 #Frequency ratio [-]
# r2 = 6/25   #Dimensionless radius of Gyration

b = 0.5     #Semi-chord
e =  -0.1    #Distance centre of mass [-] (fraction semi-chord)
a =  -0.2    #Distance torsional axis [-] (fraction semi-chord)
mu = 20     #Mass ratio [-]
sigma = 0.4 #Frequency ratio [-]
r = 0.5   #Dimensionless radius of Gyration squared r2 = IC/(mb^2)+(e-a)^2

# With mu=m/(rho*b^2), rho=1, b2=0.25 => m= 15kg
# With r2=IC/(mb^2)+(e-a) => IC=0.7875


w_theta = 100           #Twist frequency [Hz]. Used for definition of reduced velocity & normalization. 
dCLdt = 6.283185    #Lift curve slope , 2pi

n = 100                                #Number of discrete reduced velocities to evaluate 
maxvel=2.5
V = np.linspace(0.01, maxvel , n)  #Reduced velocity range
V = 2.5
#----------------------------------------------------------------------------

# Derived parameters & array initialisation
d = (e-a)           #Offset between centre of mass and torsional axis
d2 = d * d
x_theta = d
u = V*w_theta*b   #Dimensional velocity

In [52]:
# Create state space representation then solve the eigenvalue problem
    # Mass and Stiffness Matrices
M = np.array([[1, x_theta],
                [x_theta, r**2]])

K = np.array([
    [sigma**2 / V**2, 2/mu],
    [0, (r**2 / V**2) - ((2 / mu) * (a + 0.5))]])

A = np.block([
    [np.zeros_like(M), -np.eye(2)],
    [la.inv(M) @ (K), np.zeros_like(M)]
])

#eigvals, eigvecs = la.eig(A)

p,eigvecs = np.linalg.eig(A)

eigvals = p * V * w_theta

# # Remove complex conjugates, retain positive frequencies i.e., p > 0
# eigvals = eigvals[np.where(np.real(eigvals) > 0)]
# eigvecs = eigvecs[:, np.where(np.real(eigvals) > 0)]

In [53]:
# One big debug print with all the matrices and results
# print(f"Mass Matrix: {M}")
# print(f"Stiffness Matrix: {K}")
# print(f"State Space Matrix: {A}")
print(f"P Values =",p)
print(f"EigenValues =",eigvals)
print(f"Eigenvectors: {eigvecs}")

P Values = [-0.09830191+0.15164629j -0.09830191-0.15164629j  0.09830191+0.15164629j
  0.09830191-0.15164629j]
EigenValues = [-24.57547679+37.91157334j -24.57547679-37.91157334j
  24.57547679+37.91157334j  24.57547679-37.91157334j]
Eigenvectors: [[-0.93543354+0.j         -0.93543354-0.j         -0.93543354+0.j
  -0.93543354-0.j        ]
 [ 0.12472447-0.2788924j   0.12472447+0.2788924j   0.12472447+0.2788924j
   0.12472447-0.2788924j ]
 [-0.0919549 +0.14185503j -0.0919549 -0.14185503j  0.0919549 +0.14185503j
   0.0919549 -0.14185503j]
 [-0.03003234-0.04632966j -0.03003234+0.04632966j  0.03003234-0.04632966j
   0.03003234+0.04632966j]]


In [34]:
print("Reduced Velocity  = ",  V)
print("Velocity = ",u)
print("EigenValues = ",eig1[-1], eig2[-1], eig3[-1], eig4[-1])
print("EigenVectors = ",vec1[-1], vec2[-1], vec3[-1], vec4[-1])

Reduced Velocity  =  2.5
Velocity =  125.0
EigenValues =  (-24.575476790756056+37.911573336203304j) (-24.575476790756056-37.911573336203304j) (24.57547679075607+37.911573336203354j) (24.57547679075607-37.911573336203354j)
EigenVectors =  [-0.93543354+0.j          0.12472447-0.2788924j  -0.0919549 +0.14185503j
 -0.03003234-0.04632966j] [-0.93543354-0.j          0.12472447+0.2788924j  -0.0919549 -0.14185503j
 -0.03003234+0.04632966j] [-0.93543354+0.j          0.12472447+0.2788924j   0.0919549 +0.14185503j
  0.03003234-0.04632966j] [-0.93543354-0.j          0.12472447-0.2788924j   0.0919549 -0.14185503j
  0.03003234+0.04632966j]
